In [9]:
import numpy as np
from glob import glob
import cv2                
import matplotlib.pyplot as plt   
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [7]:
class Breed_Classification(object):
    
    def __init__(self, img_path):
        self.img_path = img_path
        self.img = cv2.imread(self.img_path)
        self.gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        VGG16 = models.vgg16(pretrained=True)
        self.use_cuda = torch.cuda.is_available()
        self.criterion = nn.NLLLoss()
        self.optimizer = optim.Adam(model_transfer.classifier.parameters(), lr=0.001)
        self.loaders = self.initate_data
        self.dataset = None


    def face_detector(self,img_path = None):
        """ returns ture if there a face is detected
        Note: this function takes the path of images, 
        and it has a default image if there's no provided image  
        """
        
        if gray == None:
            img_path = self.gray
        else :
            img = cv2.imread(self.img_path)
            img_path = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
        face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')
        faces = face_cascade.detectMultiScale(img_path)
        return len(faces) > 0
    
    def VGG16_predict(self, img_path= None):
        """returns the index of the highest probability
        Note: this function takes the path of images, 
        and it has a default image if there's no provided image  
        """  
        if img_path == None:
            img_path = self.img_path
        else:
            img_path = cv2.imread(self.img_path)          
        
        #Intiate vgg model
        VGG16 = models.vgg16(pretrained=True)
        if self.use_cuda:
            VGG16 = VGG16.cuda()
        #apply the model
        image = Image.open(self.img_path)
        in_transform = transforms.Compose([transforms.Resize(224),
                                           transforms.RandomResizedCrop(224),
                                           transforms.ToTensor()])
        image = in_transform(image)[:3,:,:].unsqueeze(0)
        
        if self.use_cuda:
            image= image.cuda()
        VGG16.avgpool = nn.AdaptiveAvgPool2d(1)
        output = VGG16(image)
            
        return torch.argmax(output)
    
    def dog_detector(self, img_path=None):
        """ returns True if the image is detected """
        if img_path ==None:
            img_path = self.img_path
        else:
            img_path = cv2.imread(self.img_path)
            
        prediction = self.VGG16_predict(self.img_path)
        return ((prediction <= 268) & (prediction >= 151))
    
    
    def initate_data(self, data_dir, batch_size):    
        """ it returns a dictionary of train, test, and valid data 
        Note: this function supposes that the supplied directory 
        contains a train, test, and valid folders inside it
        """
        
        data_dir = '/data/dog_images/'
        images_dir = {'train_dir':os.path.join(data_dir,'train'), 
                      'test_dir':os.path.join(data_dir,'test'),
                   'valid_dir':os.path.join(data_dir,'test') }
        
        normalization = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
        
        transformer = {
        'train': transforms.Compose( [transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),normalization]),
        'non-train': transforms.Compose([transforms.Resize(255), 
                                         transforms.CenterCrop(224),
                                         transforms.ToTensor(),normalization])}
        
        self.dataset = {'train_data':datasets.ImageFolder(images_dir['train_dir'], 
                                                          transform=transformer['train']),
                         'test_data': datasets.ImageFolder(images_dir['test_dir'],
                                                           transform=transformer['non-train']),
                         'valid_data': datasets.ImageFolder(images_dir['valid_dir'], 
                                                            transform=transformer['non-train'])}
        
        dataloader = {'trainloader': torch.utils.data.DataLoader(
                        self.dataset['train_data'],
                          batch_size=batch_size, shuffle=True),
                      
                    'testloader': torch.utils.data.DataLoader(
                        self.dataset['test_data'],
                        batch_size=batch_size, shuffle=True),
                      
                    'validloader': torch.utils.data.DataLoader(
                        self.dataset['valid_data'],
                        batch_size=batch_size, shuffle=True)}

        return dataloader
    
    
    def train(self, n_epochs, model,use_cuda, save_path, optimizer= None, criterion= None, loaders= None):
        """returns trained model"""
        
        if optimizer == None:
            optimizer=self.optimizer
        if criterion == None:
            criterion= self.criterion
        if loaders == None:
             loaders=self.loaders      
        # initialize tracker for minimum validation loss
        valid_loss_min = np.Inf 
        for epoch in range(1, n_epochs+1):
            # initialize variables to monitor training and validation loss
            train_loss = 0.0
            valid_loss = 0.0
            # train the model #
            model.train()
            for batch_idx, (data, target) in enumerate(loaders['train']):
                if self.use_cuda:
                    data, target = data.cuda(), target.cuda()
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()
                train_loss += ((1 / (batch_idx + 1)) * (loss.data - train_loss))
                
                
        # validate the model #
            model.eval()
            for batch_idx, (data, target) in enumerate(loaders['valid']):
                if self.use_cuda:
                    data, target = data.cuda(), target.cuda()
                output = model(data)
                loss = criterion(output, target)
                valid_loss += ((1 / (batch_idx + 1)) * (loss.data - valid_loss)) 
                
            print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'
                  .format(epoch, train_loss,
                          valid_loss))
            
            if valid_loss <= valid_loss_min:
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
                      .format(valid_loss_min,
                              valid_loss))
                
                torch.save(model.state_dict(),save_path)
                valid_loss_min = valid_loss
                
            return model
        
    def test(self, loaders, model, criterion, use_cuda):
        """returns accuracy and loss"""
        test_loss = 0.
        correct = 0.
        total = 0.
        
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['test']):
            if self.use_cuda:
                data, target = data.cuda(), target.cuda()
                # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            loss = criterion(output, target)
            test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
            pred = output.data.max(1, keepdim=True)[1]
            # compare predictions to true label
            correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            total += data.size(0)
            accuracy = 100. * correct / total
        print('Test Loss: {:.6f}\n'.format(test_loss))
        print('\nTest Accuracy: %2d%% (%2d/%2d)' % (100. * correct / total, correct, total))
        
        return accuracy, test_loss
        
    def vgg16_model(self):
        """returns a vgg module after fine tunning the last layer"""
        vgg16 = models.vgg16(pretrained=True)  
        for param in vgg16.parameters():
            param.requires_grad = False
        input_size = vgg16.classifier[0].in_features
        hidden_sizes = [2048, 1024]
        output_size = 133
        tuned_classifier = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                         nn.ReLU(),
                         nn.Dropout(0.25),
                         nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                         nn.ReLU(),
                         nn.Dropout(0.25),
                         nn.Linear(hidden_sizes[1], output_size),
                         nn.LogSoftmax(dim=1))
        vgg16.classifier = tuned_classifier
        if self.use_cuda:
            vgg16.cuda()
            
        return vgg16
    
    def predict_breed_transfer(self, img_path= None):
        """ return the breed type """
        class_names = [item[4:].replace("_", " ") for item in self.dataset['train_data'].classes]
        if img_path == None:
            img_path = self.img_path
            
        image = Image.open(img_path)
        in_transform = transforms.Compose([transforms.Resize(224),
                                           transforms.RandomResizedCrop(224),transforms.ToTensor()])
        image = in_transform(image)[:3,:,:].unsqueeze(0)
        vgg16 = self.vgg16_model() 
        model_transfer.load_state_dict(torch.load('model_transfer.pt'))
        vgg16.eval()  
        output= vgg16(image)
        idx = torch.argmax(output)
        breed = class_names[idx]   
        return breed
    
    
    def run_app(self, img_path= None):

        if img_path == None:
            img_path = self.img_path
            
        input_ = self.predict_breed_transfer(img_path)
        if self.face_detector(img_path):
            title = 'Our prediction is that this photo belong to human but it looks like: {}'
            .format(input_)
        elif self.dog_detector(img_path):
            title = 'This is dog of type: {}'.format(input_)
        else:
            title = 'ERROR'
        
        return title
    
    
    

        